In [ ]:
import os
import numpy as np
import cv2

# Path to the extracted FER-2013 dataset
dataset_folder = '/content/fer-2013'

# Paths to the train and test folders
train_folder = os.path.join(dataset_folder, 'train')
test_folder = os.path.join(dataset_folder, 'test')

# Initialize lists for image data and labels
train_images = []
train_labels = []
test_images = []
test_labels = []

# Define the emotion labels
emotion_labels = {
    0: 'angry',
    1: 'disgust',
    2: 'fear',
    3: 'happy',
    4: 'sad',
    5: 'neutral',
    6: 'surprise'
}

def load_images_from_folder(folder, label):
  # Initialize lists for image data and labels
  images = []
  labels = []

  # Loop through the images in each emotion folder
  for filename in os.listdir(folder):
    img_path = os.path.join(folder, filename)
    if filename.endswith('.jpg'): # Only load .jpg images
      img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
      if img is not None:
        # Resize image to 48x48 pixels (standard for FER-2013)
        img = cv2.resize(img, (48, 48))
        images.append(img)
        labels.append(label)
  return images, labels

# Load images for each emotion category (train)
for label in range(7):  # There are 7 emotion labels in FER-2013
  emotion_folder = os.path.join(train_folder, emotion_labels[label])
  images, labels = load_images_from_folder(emotion_folder, label)
  train_images.extend(images)
  train_labels.extend(labels)

# Load images for each emotion category (test)
for label in range(7):  # There are 7 emotion labels in FER-2013
  emotion_folder = os.path.join(test_folder, emotion_labels[label])
  images, labels = load_images_from_folder(emotion_folder, label)
  test_images.extend(images)
  test_labels.extend(labels)

# Convert lists to numpy arrays
X_train = np.array(train_images)
y_train = np.array(train_labels)
X_test = np.array(test_images)
y_test = np.array(test_labels)

# Normalize image data to [0, 1]
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Reshape images to include a single channel for grayscale (48x48x1)
X_train = X_train.reshape(-1, 48, 48, 1)
X_test = X_test.reshape(-1, 48, 48, 1)

# Verify the data shape
print(f"Training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Test labels shape: {y_test.shape}")


Training data shape: (28709, 48, 48, 1)
Test data shape: (7178, 48, 48, 1)
Training labels shape: (28709,)
Test labels shape: (7178,)


In [ ]:
import pandas as pd

# Load Sentiment140 CSV file
file_path = '/content/sentiment140/training.1600000.processed.noemoticon.csv'
data = pd.read_csv(file_path, encoding='ISO-8859-1', header=None)

# The dataset contaings columns:
# 0 - sentiment label (0 for negative, 4 for positive)
#

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import re

# Load Sentiment140 CSV file
file_path = '/content/sentiment140/training.1600000.processed.noemoticon.csv'
data = pd.read_csv(file_path, encoding='ISO-8859-1', header=None)

# Add proper column names
data.columns = ['sentiment', 'id', 'date', 'query', 'user', 'text']

# Clean the tweet text
def clean_text(text):
  # Remove URLS, mentions (@user), and extra spaces
  text = re.sub(r'http\S+|www\S+|https\S+', '', text)   # Remove URLS
  text = re.sub(r'@\w', '', text)   # Remove mentions (@user)
  text = re.sub(r'\s', ' ', text)   # Remove extra spaces
  return text

# Apply cleaning function
data['text'] = data['text'].apply(clean_text)

# Now, the dataset is cleaned, and you can split it into training and testing sets
X = data['text']
y = data['sentiment']

# Split into train and tests sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the first few rows of the cleaned data
print(data.head())

# Check the shape of the datasets
print(f"Training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")




   sentiment          id                          date     query  \
0          0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY   
1          0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY   
2          0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY   
3          0  1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   
4          0  1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   

              user                                               text  
0  _TheSpecialOne_  witchfoot  - A that's a bummer.  You shoulda g...  
1    scotthamilton  is upset that he can't update his Facebook by ...  
2         mattycus  enichan I dived many times for the ball. Manag...  
3          ElleCTF    my whole body feels itchy and like its on fire   
4           Karoli  ationwideclass no, it's not behaving at all. i...  
Training data shape: (1280000,)
Test data shape: (320000,)


In [31]:
import tensorflow as tf
import torch

# Check TensorFlow GPU support
print("TensorFlow version:", tf.__version__)
print("Is GPU available for TensorFlow?", tf.test.is_gpu_available())

# Check PyTorch GPU support
print("PyTorch version:", torch.__version__)
print("Is GPU available for PyTorch?", torch.cuda.is_available())

# Check the number of GPUs available (if multiple GPUs are supported)
print("Number of GPUs available:", torch.cuda.device_count())

# Display GPU details for PyTorch
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
else:
    print("No GPU found.")


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


TensorFlow version: 2.18.0
Is GPU available for TensorFlow? False
PyTorch version: 2.5.1+cu124
Is GPU available for PyTorch? False
Number of GPUs available: 0
No GPU found.


In [3]:
import tensorflow as tf
import torch

# Check TensorFlow GPU availability
print("TensorFlow version:", tf.__version__)
print("Is GPU available for TensorFlow?", tf.config.list_physical_devices('GPU'))

# Check PyTorch GPU availability
print("PyTorch version:", torch.__version__)
print("Is GPU available for PyTorch?", torch.cuda.is_available())
print("Number of GPUs available:", torch.cuda.device_count())


TensorFlow version: 2.18.0
Is GPU available for TensorFlow? [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
PyTorch version: 2.5.1+cu124
Is GPU available for PyTorch? True
Number of GPUs available: 1


In [2]:
# Check TensorFlow TPU availability
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # Detect TPUs
    print("TPU found")
except ValueError:
    print("No TPU found")


No TPU found


In [4]:
import torch
print("Is GPU available for PyTorch?", torch.cuda.is_available())
print("GPU count:", torch.cuda.device_count())
print("Current GPU:", torch.cuda.current_device())


Is GPU available for PyTorch? True
GPU count: 1
Current GPU: 0
